In [1]:
import os
import pandas as pd 
import pyodbc
import pymssql # pip install needed before import 
import sys
import sqlalchemy as sql # note- import as sql is needed for reading the sql query 


In [2]:
creds = {'un': 'SANDAGNET\\cra' , 'pw': '' }
engine = sql.create_engine('mssql+pymssql://%s:%s@sql2014b8/WS' % (creds['un'], creds['pw']))
lu21 =pd.read_sql('SELECT [OBJECTID],[LCKey],[subParcel], [parcelID], [apn8],[lu],[du],[genOwnID],[MGRA14],[regionID] FROM [WS].[gis].[LUDU2021_DRAFT_20220301]',
engine.connect())
lu21

,OBJECTID,LCKey,subParcel,parcelID,apn8,lu,du,genOwnID,MGRA14,regionID
0,1,436067,417000,4170,45257212,9101,0,500,2921,0
1,2,711183,19591300,195913,67238013,1110,1,500,6447,0
2,3,530617,226884100,2268841,,4118,0,1,22847,0
3,4,495745,24767600,247676,53532214,1110,1,500,2388,0
4,5,552843,29833000,298330,57008007,1110,1,500,1640,0
...,...,...,...,...,...,...,...,...,...,...
837693,837694,930996,67182302,671823,32909001,9202,0,10,11326,0
837694,837695,192455,68658600,686586,21645031,7603,0,500,14123,0
837695,837696,11033,69600900,696009,12420126,1000,1,500,15387,0
837696,837697,884722,66960925,669609,32907002,8003,0,42,11340,0


### QC Checks  

In [3]:
# 1. Check datatypes and value ranges 
print ("Test 1a: Checking datatypes for all columns")
print(lu21.dtypes)
print()

print ("Test 1b: Checking data range for all columns")
lu21.describe()



Test 1a: Checking datatypes for all columns
OBJECTID      int64
LCKey         int64
subParcel     int64
parcelID      int64
apn8         object
lu            int64
du            int64
genOwnID      int64
MGRA14        int64
regionID      int64
dtype: object

Test 1b: Checking data range for all columns


,OBJECTID,LCKey,subParcel,parcelID,lu,du,genOwnID,MGRA14,regionID
count,837698.000000,837698.000000,8.376980e+05,8.376980e+05,837698.000000,837698.000000,837698.000000,837698.000000,837698.0
mean,418849.500000,460894.052020,1.656991e+08,1.694375e+06,2209.686274,1.461886,438.910477,10906.273392,0.0
std,241822.727239,264064.389728,1.993311e+08,2.019031e+06,2267.198949,9.036230,162.538519,6318.861760,0.0
min,1.000000,1.000000,-2.600000e+03,-2.600000e+01,1000.000000,0.000000,1.000000,1.000000,0.0
25%,209425.250000,236496.250000,2.918782e+07,2.941442e+05,1110.000000,1.000000,500.000000,5739.000000,0.0
50%,418849.500000,461719.500000,5.980485e+07,6.021655e+05,1110.000000,1.000000,500.000000,10521.000000,0.0
75%,628273.750000,684091.750000,2.467686e+08,2.576474e+06,1200.000000,1.000000,500.000000,16207.000000,0.0
max,837698.000000,931003.000000,9.005027e+08,9.009444e+06,9700.000000,1848.000000,500.000000,23220.000000,0.0


In [4]:
# 2. Check null values in columns 
print("Test 2: Checking null values across all columns")
print(lu21.isnull().sum(axis = 0))

Test 2: Checking null values across all columns
OBJECTID     0
LCKey        0
subParcel    0
parcelID     0
apn8         0
lu           0
du           0
genOwnID     0
MGRA14       0
regionID     0
dtype: int64


In [5]:
# 3. Check duplicates across columns 
print("Test 3: Checking duplicates across columns")
print("Length of the dataframe=" + str(lu21.shape[0]))
print("Unique Records in each column:")
print(lu21.nunique())


Test 3: Checking duplicates across columns
Length of the dataframe=837698
Unique Records in each column:
OBJECTID     837698
LCKey        837698
subParcel    835446
parcelID     821690
apn8         742378
lu              104
du              369
genOwnID         21
MGRA14        23191
regionID          1
dtype: int64


In [6]:
#4. No DU on selected land uses

list= [1400, 1409, 1500, 1501, 1502, 9101, 4101, 4102, 4103, 4104, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118,
       4119, 4120, 6101, 7600, 7601, 7603, 7604, 7605, 7606, 7607, 7609, 9201, 9202]

# lu21[lu21.lu.isin (list)][lu21.du>0 ]
lu21_list= lu21[lu21.lu.isin (list)]
lu21_list[lu21_list['du'] > 0]

,OBJECTID,LCKey,subParcel,parcelID,apn8,lu,du,genOwnID,MGRA14,regionID
129255,129256,488783,4688100,46881,54142309,6101,1,500,1972,0
375688,375689,855848,28593200,285932,48017403,7601,1,10,9405,0
620359,620360,889329,531159202,5311592,41001020,7603,3,43,20650,0
695836,695837,855846,32906600,329066,48017402,7601,1,10,9405,0
708964,708965,886232,531309300,5313093,64352060,7607,6,10,8495,0
745789,745790,5218,51463800,514638,10552078,7601,1,20,15667,0
746815,746816,145456,67395504,673955,11019017,7603,1,20,16055,0
750793,750794,823698,58839304,588393,41111006,7603,13,43,20670,0
757887,757888,543579,62515500,625155,59805003,7603,2,33,22523,0
766012,766013,861221,65695903,656959,24305001,7603,2,10,20851,0


In [7]:
# Checking which lu have 0 du's and creating cross-tabs by lu codes and mgra 
lu_zero= lu21[lu21['du']==0]
lu_ct= pd.crosstab(index=lu_zero['lu'], columns='count')
lu_mgra= pd.crosstab(index=lu_zero['MGRA14'], columns='count')

# visualize this in ArcGIS


In [8]:
# Comparing du or lu codes between parcel and subParcel between lu 2021 and lu 2020 

